In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- 1. AYARLAR VE FONKSİYONLAR ---
# Grafiklerin notebook içinde görünmesi için
%matplotlib inline
# Bilimsel gösterim ayarları
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

def clean_and_load_hwinfo(filepath):
    """
    HWInfo CSV dosyasını okur, boolean dönüşümlerini yapar ve 
    DateTime indekslemesi oluşturur.
    """
    if not os.path.exists(filepath):
        print(f"HATA: Dosya bulunamadı -> {filepath}")
        return None

    df = pd.read_csv(filepath, low_memory=False, on_bad_lines='skip')
    df.columns = [c.strip() for c in df.columns] # Boşluk temizliği
    
    # Boolean Dönüşüm
    bool_mapping = {
        'Evet': True, 'Yes': True, 
        'Hayır': False, 'No': False, 
        'Yok': False, 'None': False,
        'Var': True
    }
    
    for col in df.columns:
        if df[col].dtype == 'object':
            unique_vals = set(df[col].dropna().unique())
            if unique_vals.issubset(set(bool_mapping.keys())):
                df[col] = df[col].map(bool_mapping)
    
    # Tarih/Saat Birleştirme
    if 'Date' in df.columns and 'Time' in df.columns:
        try:
            df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], dayfirst=True)
            df.set_index('Datetime', inplace=True)
        except Exception as e:
            print(f"Tarih formatlama hatası ({filepath}): {e}")
            
    # Sayısal Dönüşüm
    cols_to_exclude = ['Date', 'Time']
    for col in df.columns:
        if col not in cols_to_exclude and df[col].dtype == 'object':
            if not pd.api.types.is_bool_dtype(df[col]):
                 df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

def compare_ram_consistency(data_dict):
    """
    Profiller arası RAM zamanlamalarını karşılaştırır.
    """
    timing_cols = ['Tcas [T]', 'Trcd [T]', 'Trp [T]', 'Tras [T]', 'Trc [T]', 'Trfc [T]', 'Command Rate [T]']
    
    if not data_dict:
        print("Analiz edilecek veri yok.")
        return

    first_df = next(iter(data_dict.values()))
    valid_cols = [c for c in timing_cols if c in first_df.columns]
    
    print("--- RAM Zamanlamaları Tutarlılık Analizi ---")
    timing_summary = {}
    
    for name, df in data_dict.items():
        if df is not None:
            timings = df[valid_cols].iloc[0].to_dict()
            timing_summary[name] = timings

    timing_df = pd.DataFrame(timing_summary)
    is_consistent = (timing_df.nunique(axis=1) == 1).all()
    
    if is_consistent:
        print("ONAY: Tüm profillerde RAM zamanlamaları birebir aynıdır.")
        display(timing_df.iloc[:, [0]])
    else:
        print("UYARI: RAM zamanlamalarında farklılık var!")
        display(timing_df)

def plot_thermal_power_analysis(data_dict):
    """
    Sıcaklık ve Güç grafikleri ve özet istatistikler.
    """
    if not data_dict:
        return

    col_temp = 'CPU (Tctl/Tdie) [°C]'
    col_power = 'CPU Tüm Güç Tüketimi [W]'
    
    fig, axes = plt.subplots(2, 1, figsize=(15, 12), sharex=True)
    sns.set_style("darkgrid")
    
    stats_list = []

    for name, df in data_dict.items():
        if df is None: continue
            
        start_time = df.index[0]
        seconds = (df.index - start_time).total_seconds()
        
        axes[0].plot(seconds, df[col_temp], label=name, linewidth=1.5, alpha=0.8)
        axes[1].plot(seconds, df[col_power], label=name, linewidth=1.5, alpha=0.8)
        
        stats_list.append({
            'Profil': name,
            'Ort. Sıcaklık (°C)': df[col_temp].mean(),
            'Max Sıcaklık (°C)': df[col_temp].max(),
            'Ort. Güç (W)': df[col_power].mean(),
            'Max Güç (W)': df[col_power].max()
        })
    
    axes[0].set_title('CPU Sıcaklığı (Tctl/Tdie)', fontsize=14)
    axes[0].set_ylabel('Sıcaklık [°C]', fontsize=12)
    axes[0].legend()
    axes[0].axhline(y=95, color='r', linestyle='--', alpha=0.5, label='95°C Limit') 
    
    axes[1].set_title('CPU Paket Güç Tüketimi', fontsize=14)
    axes[1].set_ylabel('Güç [W]', fontsize=12)
    axes[1].set_xlabel('Süre [Saniye]', fontsize=12)
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    print("\n--- Termal ve Güç İstatistikleri Özeti ---")
    stats_df = pd.DataFrame(stats_list).set_index('Profil')
    display(stats_df.style.background_gradient(cmap='RdYlGn_r', subset=['Ort. Sıcaklık (°C)', 'Max Sıcaklık (°C)', 'Ort. Güç (W)'])
                   .format("{:.2f}"))

# --- 2. VERİ YÜKLEME ---
files = {
    "Default_Optimized": "Fans ve apu down.csv",
    "PBO_95W_Neg30_Plus200": "-30 CCD PBO 95 +200 mhz.csv",
    "PBO_65W_Neg25": "65W-85W_75A_150A-25_Per_CCD.csv",
    "PBO_95W_Neg20": "95W-115W_80A_140A-20_Per_CCD.csv"
}

dataframes = {}
print("Veriler yükleniyor...")
for name, path in files.items():
    loaded_df = clean_and_load_hwinfo(path)
    if loaded_df is not None:
        dataframes[name] = loaded_df
        print(f"[{name}] yüklendi. ({len(loaded_df)} satır)")

# --- 3. ANALİZİ ÇALIŞTIR ---
compare_ram_consistency(dataframes)
plot_thermal_power_analysis(dataframes)

In [ ]:
def plot_clock_efficiency_analysis(data_dict):
    """
    Çekirdek saat hızlarını, Clock Stretching durumunu ve Watt başına performansı analiz eder.
    """
    if not data_dict:
        return

    # Sütun İsimleri (HWInfo)
    col_core_clock = 'Çekirdek Saatler (avg) [MHz]'
    col_eff_clock = 'Çekirdek Etkili Saatler (avg) [MHz]'
    col_power = 'CPU Tüm Güç Tüketimi [W]'
    
    # Renk Paleti
    colors = sns.color_palette("tab10", n_colors=len(data_dict))
    
    # 3 Alt Grafik Oluştur (Saatler, Stretching, Verimlilik)
    fig, axes = plt.subplots(3, 1, figsize=(15, 18), sharex=True)
    sns.set_style("darkgrid")
    
    summary_stats = []
    
    for i, (name, df) in enumerate(data_dict.items()):
        if df is None: continue
            
        # Zaman ekseni (Saniye)
        start_time = df.index[0]
        seconds = (df.index - start_time).total_seconds()
        
        # 1. Grafik: Effective Clock (Gerçek Performans)
        # Core Clock yerine Effective Clock'a odaklanıyoruz çünkü gerçek işi bu yapar.
        axes[0].plot(seconds, df[col_eff_clock], label=f"{name}", linewidth=1.5, color=colors[i])
        
        # 2. Grafik: Clock Stretching (Saat Esnemesi)
        # (Core Clock - Effective Clock) farkı. İdealde 0'a yakın olmalı.
        # Yük altında biraz fark normaldir ama büyük farklar "Yalancı Frekans" demektir.
        stretching = df[col_core_clock] - df[col_eff_clock]
        # Negatif değerleri 0'a çek (bazen sensör hatası olabilir)
        stretching = stretching.clip(lower=0)
        axes[1].plot(seconds, stretching, label=name, linewidth=1.5, color=colors[i], alpha=0.8)
        
        # 3. Grafik: Anlık Verimlilik (MHz per Watt)
        # Formül: Effective Clock / Power Consumption
        # 0'a bölme hatasını önlemek için power'ı filtrele
        power_safe = df[col_power].replace(0, 1) 
        efficiency = df[col_eff_clock] / power_safe
        axes[2].plot(seconds, efficiency, label=name, linewidth=1.5, color=colors[i])
        
        # İstatistik Topla
        summary_stats.append({
            'Profil': name,
            'Ort. Core Clock (MHz)': df[col_core_clock].mean(),
            'Ort. Eff. Clock (MHz)': df[col_eff_clock].mean(),
            'Max Eff. Clock (MHz)': df[col_eff_clock].max(),
            'Ort. Stretching (MHz)': stretching.mean(), # Düşük olması iyi
            'Verimlilik (MHz/Watt)': efficiency.mean()  # Yüksek olması iyi
        })

    # Düzenlemeler
    axes[0].set_title('Ortalama Çekirdek "Effective" Saat Hızı (Performans)', fontsize=14)
    axes[0].set_ylabel('Frekans [MHz]', fontsize=12)
    axes[0].legend(loc='lower right')
    
    axes[1].set_title('Clock Stretching (Frekans Kaybı) Analizi', fontsize=14)
    axes[1].set_ylabel('Kayıp Frekans [MHz]', fontsize=12)
    axes[1].legend()
    # Kritik not ekle
    axes[1].text(0.01, 0.9, 'Düşük değer daha iyidir (Stabilite Göstergesi)', transform=axes[1].transAxes, fontsize=10, bbox=dict(facecolor='white', alpha=0.8))

    axes[2].set_title('Verimlilik Eğrisi (Watt Başına MHz)', fontsize=14)
    axes[2].set_ylabel('MHz / Watt', fontsize=12)
    axes[2].set_xlabel('Test Süresi [Saniye]', fontsize=12)
    axes[2].legend()

    plt.tight_layout()
    plt.show()
    
    # Özet Tablo
    print("\n--- Frekans ve Verimlilik İstatistikleri ---")
    stat_df = pd.DataFrame(summary_stats).set_index('Profil')
    
    # Renklendirme: 
    # Eff Clock ve Verimlilik -> Yüksek iyi (Koyu Yeşil)
    # Stretching -> Düşük iyi (Tersi, Koyu Yeşil düşük değerde olacak)
    display(stat_df.style.background_gradient(cmap='RdYlGn', subset=['Ort. Eff. Clock (MHz)', 'Max Eff. Clock (MHz)', 'Verimlilik (MHz/Watt)'])
                         .background_gradient(cmap='RdYlGn_r', subset=['Ort. Stretching (MHz)']) # Düşük stretching yeşil olur
                         .format("{:.2f}"))

plot_clock_efficiency_analysis(dataframes)

In [ ]:
def plot_voltage_limits_analysis(data_dict):
    """
    Voltaj davranışını, V-F (Voltaj-Frekans) eğrisini ve
    işlemcinin hangi limitlere (PPT, TDC, EDC, Termal) takıldığını analiz eder.
    """
    if not data_dict:
        return

    # Sütun Tanımları
    col_voltage = 'Core VIDs (avg) [V]'
    col_eff_clock = 'Çekirdek Etkili Saatler (avg) [MHz]'
    # Limit Sütunları (Yüzdesel)
    col_ppt_limit = 'CPU PPT Limit [%]'
    col_edc_limit = 'CPU EDC Sınırı [%]'
    col_tdc_limit = 'CPU TDC Sınırı [%]'
    col_thermal_limit = 'Thermal Limit [%]' # Bazen yüzde olmayabilir, HWInfo sürümüne göre değişir
    
    # Renkler
    colors = sns.color_palette("tab10", n_colors=len(data_dict))
    
    # 2 Satır, 2 Sütunlu Grid
    fig, axes = plt.subplots(2, 2, figsize=(18, 14))
    sns.set_style("darkgrid")
    
    # Düzleştirelim kolay erişim için
    ax_volt = axes[0, 0]
    ax_vf = axes[0, 1]
    ax_limits = axes[1, 0]
    ax_limit_dist = axes[1, 1] # Limit dağılımı
    
    limit_stats = []

    for i, (name, df) in enumerate(data_dict.items()):
        if df is None: continue
        
        # Zaman ekseni
        seconds = (df.index - df.index[0]).total_seconds()
        
        # 1. Voltaj Zaman Serisi
        ax_volt.plot(seconds, df[col_voltage], label=name, color=colors[i], alpha=0.8)
        
        # 2. V-F Curve (Scatter Plot) - EN KRİTİK GRAFİK
        # Çok fazla nokta olacağı için alpha'yı düşük, nokta boyutunu küçük tutuyoruz
        # Her 10. veriyi alarak (sample) grafiği hafifletelim
        sampled_df = df.iloc[::10, :]
        sns.scatterplot(x=sampled_df[col_voltage], y=sampled_df[col_eff_clock], 
                        ax=ax_vf, label=name, color=colors[i], alpha=0.5, s=15)
        
        # 3. Limit Kullanımı (Hangi limit %100'e vuruyor?)
        # Burada sadece PPT, EDC ve TDC ortalamalarını bar chart olarak gösterelim
        avg_ppt = df[col_ppt_limit].mean()
        avg_edc = df[col_edc_limit].mean()
        avg_tdc = df[col_tdc_limit].mean()
        
        limit_stats.append({
            'Profil': name,
            'PPT Kullanımı (%)': avg_ppt,
            'EDC Kullanımı (%)': avg_edc,
            'TDC Kullanımı (%)': avg_tdc,
            'Ort. Voltaj (V)': df[col_voltage].mean()
        })

    # Limit İstatistiklerini DataFrame yap
    limit_df = pd.DataFrame(limit_stats).set_index('Profil')

    # Grafik 1 Ayarları
    ax_volt.set_title('Çekirdek Voltajı (VID) Zaman Serisi', fontsize=14)
    ax_volt.set_ylabel('Voltaj [V]', fontsize=12)
    ax_volt.legend()
    
    # Grafik 2 Ayarları (V-F Curve)
    ax_vf.set_title('V-F Eğrisi (Verimlilik Analizi)', fontsize=14)
    ax_vf.set_xlabel('Voltaj [V]', fontsize=12)
    ax_vf.set_ylabel('Etkili Saat Hızı [MHz]', fontsize=12)
    ax_vf.legend()
    ax_vf.grid(True, which='both', linestyle='--', linewidth=0.5)
    # İdeal bölge notu
    ax_vf.text(0.05, 0.95, 'Sol Üst Köşe = Daha İyi Silikon/Ayar', transform=ax_vf.transAxes, 
               fontsize=10, bbox=dict(facecolor='white', alpha=0.8))
    
    # Grafik 3: Limit Karşılaştırması (Bar Chart)
    limit_df[['PPT Kullanımı (%)', 'EDC Kullanımı (%)', 'TDC Kullanımı (%)']].plot(kind='bar', ax=ax_limits, width=0.8)
    ax_limits.set_title('Ortalama Limit Kullanım Yüzdeleri', fontsize=14)
    ax_limits.set_ylabel('Kullanım [%]', fontsize=12)
    ax_limits.axhline(y=100, color='r', linestyle='--', label='Limit Sınırı')
    ax_limits.legend(loc='lower right')
    ax_limits.tick_params(axis='x', rotation=15)

    # Grafik 4: Isı Haritası (Voltaj vs Sıcaklık) 
    # Hangi voltajda ne kadar ısınıyoruz?
    # Sadece ilk ve son profili karşılaştırmak karmaşayı önler, ama burada
    # En agresif profil (-30 CCD) ile Default'u karşılaştıralım (Varsa)
    # Ya da tüm veriyi tek bir scatter ile gösterelim: X=Voltaj, Y=Sıcaklık
    for i, (name, df) in enumerate(data_dict.items()):
         sampled_df = df.iloc[::20, :] # Daha seyrek örneklem
         ax_limit_dist.scatter(sampled_df[col_voltage], sampled_df['CPU (Tctl/Tdie) [°C]'], 
                               label=name, color=colors[i], alpha=0.4)
    
    ax_limit_dist.set_title('Voltaj vs Sıcaklık İlişkisi', fontsize=14)
    ax_limit_dist.set_xlabel('Voltaj [V]', fontsize=12)
    ax_limit_dist.set_ylabel('Sıcaklık [°C]', fontsize=12)
    ax_limit_dist.legend()

    plt.tight_layout()
    plt.show()
    
    # Tablo Gösterimi
    print("\n--- Güç Limitleri ve Voltaj Özeti ---")
    display(limit_df.style.background_gradient(cmap='YlOrRd', vmin=50, vmax=100)
                    .format("{:.2f}"))

plot_voltage_limits_analysis(dataframes)

In [ ]:
import numpy as np
from math import pi

def create_summary_and_radar_chart(data_dict, baseline_name="Default_Optimized"):
    """
    Tüm metrikleri özetler, baz profil (baseline) ile karşılaştırır 
    ve Radar (Örümcek) grafiği çizer.
    """
    if not data_dict:
        return

    # Metrikler için sütun eşleştirmeleri
    cols = {
        'Effective Clock': 'Çekirdek Etkili Saatler (avg) [MHz]',
        'Power (Watts)': 'CPU Tüm Güç Tüketimi [W]',
        'Temperature': 'CPU (Tctl/Tdie) [°C]',
        'Voltage': 'Core VIDs (avg) [V]'
    }
    
    summary_data = []

    # 1. Ham Verileri Topla
    for name, df in data_dict.items():
        if df is None: continue
        
        row = {'Profil': name}
        for metric, col_name in cols.items():
            row[metric] = df[col_name].mean()
            
        # Verimlilik (Efficiency) = Clock / Power
        row['Efficiency (MHz/W)'] = row['Effective Clock'] / row['Power (Watts)']
        summary_data.append(row)
    
    summary_df = pd.DataFrame(summary_data).set_index('Profil')
    
    # 2. Normalizasyon (Baseline'a göre oranla)
    # Eğer baseline ismi tam tutmazsa, ilk sıradakini baseline yap
    if baseline_name not in summary_df.index:
        baseline_name = summary_df.index[0]
        print(f"UYARI: '{baseline_name}' referans alındı.")
    
    baseline_vals = summary_df.loc[baseline_name]
    
    # Normalizasyon Formülü: (Değer / Baseline) * 100
    norm_df = (summary_df / baseline_vals) * 100
    
    # DİKKAT: Sıcaklık ve Güç için "Düşük Olan İyidir".
    # Grafikte "Dışarıya Doğru" olanın iyi görünmesi için bunları ters çevirelim (Invert logic for visuals)
    # Yani: Eğer güç %90 ise (daha az tüketiyorsa), grafikte %110 gibi pozitif algılanmalı.
    # Görsel Puan = 200 - Yüzde Değer (Basit bir tersleme)
    # Ancak bilimsel doğruluk için radar grafikte eksenleri ters çevirmek daha iyidir.
    # Biz burada standart yüzdeleri tablo olarak, "Skor"ları grafik olarak hazırlayalım.
    
    print(f"--- '{baseline_name}' Referans Alınarak Oluşturulan Fark Tablosu (%) ---")
    # Okunabilir fark tablosu (Örn: 105% -> %5 artış)
    display(norm_df.style.background_gradient(cmap='RdYlGn', subset=['Effective Clock', 'Efficiency (MHz/W)'])
                         .background_gradient(cmap='RdYlGn_r', subset=['Power (Watts)', 'Temperature', 'Voltage']) # Düşük iyi
                         .format("{:.1f}%"))

    # 3. Radar Grafiği Hazırlığı
    # Grafikte göstermek istediğimiz özellikler
    categories = ['Effective Clock', 'Efficiency (MHz/W)', 'Soğutma Performansı', 'Güç Tasarrufu']
    N = len(categories)
    
    # Verileri hazırla (Soğutma ve Güç için ters orantı kullanacağız: 100/Değer * 100)
    # Böylece düşük sıcaklık -> Yüksek puan olur.
    radar_data = {}
    for name in summary_df.index:
        vals = []
        # Clock (Yüksek iyi)
        vals.append((summary_df.loc[name, 'Effective Clock'] / baseline_vals['Effective Clock']) * 100)
        # Efficiency (Yüksek iyi)
        vals.append((summary_df.loc[name, 'Efficiency (MHz/W)'] / baseline_vals['Efficiency (MHz/W)']) * 100)
        # Soğutma (Düşük sıcaklık -> Yüksek puan)
        vals.append((baseline_vals['Temperature'] / summary_df.loc[name, 'Temperature']) * 100)
        # Güç Tasarrufu (Düşük Watt -> Yüksek puan)
        vals.append((baseline_vals['Power (Watts)'] / summary_df.loc[name, 'Power (Watts)'] * 100))
        
        radar_data[name] = vals

    # Radar Çizimi
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1] # Kapatmak için
    
    plt.figure(figsize=(10, 10))
    ax = plt.subplot(111, polar=True)
    
    # Eksen etiketleri
    plt.xticks(angles[:-1], categories, color='black', size=12)
    
    # Y Eksen aralıkları
    ax.set_rlabel_position(0)
    plt.yticks([80, 100, 120], ["80%", "100%", "120%"], color="grey", size=10)
    plt.ylim(70, 130) # Grafiğin odaklanacağı aralık (Gerekirse ayarlanabilir)
    
    colors = sns.color_palette("tab10", n_colors=len(data_dict))
    
    for i, (name, vals) in enumerate(radar_data.items()):
        vals += vals[:1]
        ax.plot(angles, vals, linewidth=2, linestyle='solid', label=name, color=colors[i])
        ax.fill(angles, vals, color=colors[i], alpha=0.1)
    
    plt.title(f"Performans ve Verimlilik Dengesi\n(Referans: {baseline_name} = 100%)", size=15, y=1.1)
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    
    plt.show()

create_summary_and_radar_chart(dataframes)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def plot_distribution_and_core_deltas(data_dict):
    """
    Verilerin istatistiksel dağılımını (Box Plot) ve 
    çekirdekler arası sıcaklık farklarını (Core Delta) analiz eder.
    HATA DÜZELTME: Veriler 'Long Format'a çevrilerek Duplicate Index hatası giderilmiştir.
    """
    if not data_dict:
        return

    # Sütun Tanımları
    col_temp = 'CPU (Tctl/Tdie) [°C]'
    col_eff_clock = 'Çekirdek Etkili Saatler (avg) [MHz]'
    col_soc_power = 'CPU SoC Güç Tüketimi (SVI3 TFN) [W]'
    
    # Çekirdek sıcaklık sütunlarını bulalım
    first_df = next(iter(data_dict.values()))
    core_temp_cols = [c for c in first_df.columns if 'Core' in c and 'CCD' in c and '[°C]' in c]
    
    # --- VERİ HAZIRLIĞI (LONG FORMAT) ---
    # Yan yana birleştirme yerine, verileri etiketleyip alt alta dizeceğiz.
    temp_data = []
    clock_data = []
    soc_data = []
    delta_list = []

    for name, df in data_dict.items():
        if df is None: continue
        
        # 1. Sıcaklık Verisi
        t_df = df[[col_temp]].copy()
        t_df.columns = ['Value']
        t_df['Profil'] = name
        temp_data.append(t_df)
        
        # 2. Saat Hızı Verisi
        c_df = df[[col_eff_clock]].copy()
        c_df.columns = ['Value']
        c_df['Profil'] = name
        clock_data.append(c_df)
        
        # 3. SoC Güç Verisi
        s_df = df[[col_soc_power]].copy()
        s_df.columns = ['Value']
        s_df['Profil'] = name
        soc_data.append(s_df)
        
        # 4. Core Delta Hesabı (Satır bazlı olduğu için index sorunu yaratmaz)
        if core_temp_cols:
            # Sadece sayısal sütunları alalım
            core_temps = df[core_temp_cols].apply(pd.to_numeric, errors='coerce')
            max_core = core_temps.max(axis=1)
            min_core = core_temps.min(axis=1)
            delta = max_core - min_core
            delta_list.append(pd.DataFrame({'Delta': delta, 'Profil': name}))

    # Listeleri DataFrame'e çevir (Index'i resetleyerek birleştiriyoruz, hata önleniyor)
    long_temp = pd.concat(temp_data).reset_index(drop=True)
    long_clock = pd.concat(clock_data).reset_index(drop=True)
    long_soc = pd.concat(soc_data).reset_index(drop=True)
    
    # Renk Paleti
    colors = sns.color_palette("tab10", n_colors=len(data_dict))
    
    # 2x2 Grafik Düzeni
    fig, axes = plt.subplots(2, 2, figsize=(18, 14))
    sns.set_style("whitegrid")
    
    # Grafik 1: Sıcaklık Dağılımı (Box Plot)
    sns.boxplot(data=long_temp, x='Profil', y='Value', hue='Profil', legend=False, ax=axes[0, 0], palette="tab10")
    axes[0, 0].set_title('Sıcaklık Kararlılık Dağılımı (Box Plot)', fontsize=14)
    axes[0, 0].set_ylabel('Sıcaklık [°C]', fontsize=12)
    axes[0, 0].axhline(y=95, color='r', linestyle='--', alpha=0.5, label='Limit')
    axes[0, 0].tick_params(axis='x', rotation=15)

    # Grafik 2: Effective Clock Dağılımı (Box Plot)
    sns.boxplot(data=long_clock, x='Profil', y='Value', hue='Profil', legend=False, ax=axes[0, 1], palette="tab10")
    axes[0, 1].set_title('Saat Hızı Kararlılık Dağılımı', fontsize=14)
    axes[0, 1].set_ylabel('Frekans [MHz]', fontsize=12)
    axes[0, 1].tick_params(axis='x', rotation=15)
    
    # Grafik 3: Core Delta (KDE Plot)
    if delta_list:
        long_delta = pd.concat(delta_list).reset_index(drop=True)
        sns.kdeplot(data=long_delta, x='Delta', hue='Profil', legend=False, ax=axes[1, 0], fill=True, alpha=0.3, linewidth=2, palette="tab10")
        axes[1, 0].set_title('Çekirdekler Arası Sıcaklık Farkı (Delta T) Yoğunluğu', fontsize=14)
        axes[1, 0].set_xlabel('Sıcaklık Farkı [°C]', fontsize=12)
        axes[1, 0].set_ylabel('Gözlem Sıklığı', fontsize=12)
    
    # Grafik 4: SoC Güç Tüketimi (Violin Plot)
    sns.violinplot(data=long_soc, x='Profil', y='Value', hue='Profil', legend=False, ax=axes[1, 1], palette="tab10", inner="quartile")
    axes[1, 1].set_title('SoC Güç Tüketimi Dağılımı (RAM Etkisi)', fontsize=14)
    axes[1, 1].set_ylabel('Güç [W]', fontsize=12)
    axes[1, 1].tick_params(axis='x', rotation=15)

    plt.tight_layout()
    plt.show()
    
    # İstatistik Tablosunu Göster
    if delta_list:
        print("\n--- Çekirdekler Arası Termal Dengesizlik (Delta T) Özeti ---")
        stats = long_delta.groupby('Profil')['Delta'].agg(['mean', 'max', 'min']).rename(columns={'mean': 'Ortalama Delta', 'max': 'Max Delta', 'min': 'Min Delta'})
        display(stats.style.background_gradient(cmap='RdYlGn_r').format("{:.2f}"))

plot_distribution_and_core_deltas(dataframes)

In [ ]:
def analyze_core_quality(data_dict):
    """
    Her bir çekirdeğin (Core 0-5) performansını analiz ederek
    'Golden Core' (En İyi) ve 'Weak Core' (En Zayıf) tespiti yapar.
    """
    if not data_dict:
        return

    # Renk Paleti
    colors = sns.color_palette("viridis", n_colors=6) # 6 Çekirdek için
    
    # Veri Toplama
    core_stats = []
    
    # HWInfo Sütun İsimlerini Tespit Etme (Dinamik)
    first_df = next(iter(data_dict.values()))
    
    # Core Clock Sütunları (Effective Clock tercih edilir)
    # Örn: "Core 0 T0 Etkili Saat" ve "Core 0 T1 Etkili Saat" ortalaması
    # Veya direkt "Core 0 VID" vb.
    # HWInfo'da genelde "Core 0 T0 Etkili Saat [MHz]" vardır.
    
    print("--- Çekirdek Kalite Analizi Başlatılıyor ---")
    
    for name, df in data_dict.items():
        if df is None: continue
        
        for i in range(6): # Ryzen 7600X 6 Çekirdekli
            # Sütun isimlerini bulmaya çalış
            # Efektif Saat (T0 ve T1 ortalaması veya sadece T0)
            eff_col_t0 = f"Core {i} T0 Etkili Saat [MHz]"
            eff_col_t1 = f"Core {i} T1 Etkili Saat [MHz]"
            
            # Sıcaklık (Core Temp)
            # Format: "Core0 (CCD1) [°C]" veya "Core 0 (CCD1) [°C]"
            # Regex yerine basit döngüyle bulalım
            temp_col = None
            for c in df.columns:
                if f"Core{i}" in c and "CCD" in c and "[°C]" in c: # Core0 bitişik
                    temp_col = c
                    break
                if f"Core {i}" in c and "CCD" in c and "[°C]" in c: # Core 0 ayrı
                    temp_col = c
                    break
            
            if eff_col_t0 in df.columns and temp_col:
                # Saat hızı (T0 ve varsa T1 ortalaması)
                if eff_col_t1 in df.columns:
                    avg_clock = (df[eff_col_t0] + df[eff_col_t1]) / 2
                else:
                    avg_clock = df[eff_col_t0]
                
                avg_clock_val = avg_clock.mean()
                avg_temp_val = df[temp_col].mean()
                
                core_stats.append({
                    'Profil': name,
                    'Core Index': f"Core {i}",
                    'Ortalama Frekans (MHz)': avg_clock_val,
                    'Ortalama Sıcaklık (°C)': avg_temp_val,
                    # Kalite Skoru: Birim sıcaklık başına frekans (Basit metrik)
                    'Kalite Skoru': avg_clock_val / avg_temp_val
                })

    if not core_stats:
        print("HATA: Çekirdek sütunları bulunamadı.")
        return

    stats_df = pd.DataFrame(core_stats)
    
    # GÖRSELLEŞTİRME
    fig, axes = plt.subplots(2, 1, figsize=(15, 12))
    sns.set_style("whitegrid")
    
    # Grafik 1: Her Profilde Çekirdek Frekansları
    sns.barplot(data=stats_df, x='Core Index', y='Ortalama Frekans (MHz)', hue='Profil', ax=axes[0], palette="tab10")
    axes[0].set_title('Çekirdek Bazlı Ortalama Frekanslar (Yüksek İyi)', fontsize=14)
    axes[0].set_ylim(stats_df['Ortalama Frekans (MHz)'].min() - 50, stats_df['Ortalama Frekans (MHz)'].max() + 50)
    axes[0].legend(loc='lower right')
    
    # Grafik 2: Isı Haritası (Hangi Çekirdek Daha Sıcak?)
    # Pivot yaparak Heatmap formatına getirelim: Satırlar=Profil, Sütunlar=Core
    heatmap_data = stats_df.pivot(index='Profil', columns='Core Index', values='Ortalama Sıcaklık (°C)')
    sns.heatmap(heatmap_data, annot=True, fmt=".1f", cmap="RdYlGn_r", ax=axes[1], linewidths=.5)
    axes[1].set_title('Çekirdek Sıcaklık Haritası (Düşük İyi)', fontsize=14)
    
    plt.tight_layout()
    plt.show()
    
    # EN İYİ VE EN KÖTÜ ÇEKİRDEĞİ BULMA (Tüm Profillerin Ortalaması)
    avg_per_core = stats_df.groupby('Core Index')[['Ortalama Frekans (MHz)', 'Ortalama Sıcaklık (°C)', 'Kalite Skoru']].mean()
    best_core = avg_per_core['Kalite Skoru'].idxmax()
    worst_core = avg_per_core['Kalite Skoru'].idxmin()
    
    print(f"\n--- SİLİKON ANALİZ SONUCU (GENEL) ---")
    print(f"🏆 EN İYİ ÇEKİRDEK (Golden Core): {best_core}")
    print(f"⚠️ EN ZAYIF ÇEKİRDEK (Weak Core): {worst_core}")
    print("\nDetaylı Ortalama Veriler:")
    display(avg_per_core.sort_values('Kalite Skoru', ascending=False).style.background_gradient(cmap='RdYlGn', subset=['Kalite Skoru']))

analyze_core_quality(dataframes)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

def analyze_system_health_and_cooling(data_dict):
    """
    Adım 11 ve 12: Anakart VRM, PSU Kararlılığı ve Fan/Soğutma Fiziği Analizi.
    """
    if not data_dict:
        return

    # --- Sütun Tanımları ---
    # VRM: Genelde 'VR MOS' veya 'SVI3 TFN VRM'
    # İlk df'den kontrol edelim
    first_df = next(iter(data_dict.values()))
    
    col_vrm_candidates = ['CPU VDDCR_VDD VRM (SVI3 TFN) [°C]', 'MOS [°C]', 'VR MOS [°C]']
    col_vrm = next((c for c in col_vrm_candidates if c in first_df.columns), None)
    
    col_12v = '+12V [V]'
    col_rpm = 'CPU [RPM]'
    col_temp = 'CPU (Tctl/Tdie) [°C]'
    
    if not col_vrm or col_12v not in first_df.columns:
        print(f"HATA: Kritik sütunlar eksik. (VRM veya +12V)")
        return

    print(f"Analiz Sütunları:\n- VRM: {col_vrm}\n- PSU: {col_12v}\n- Fan: {col_rpm}")

    # Renk Paleti
    colors = sns.color_palette("tab10", n_colors=len(data_dict))
    
    # 2x2 Grafik Grid'i
    fig, axes = plt.subplots(2, 2, figsize=(18, 14))
    sns.set_style("darkgrid")
    
    stats_list = []
    
    for i, (name, df) in enumerate(data_dict.items()):
        if df is None: continue
        
        # Zaman ekseni oluşturma (Index tipine karşı dayanıklı)
        seconds = None
        if isinstance(df.index, pd.DatetimeIndex):
            seconds = (df.index - df.index[0]).total_seconds()
        elif 'Datetime' in df.columns:
            try:
                dt = pd.to_datetime(df['Datetime'])
                seconds = (dt - dt.iloc[0]).dt.total_seconds()
            except: pass
            
        if seconds is None:
            seconds = df.index # Fallback: Örnekleme sayısı
            x_label = "Örnekleme (Sample)"
        else:
            x_label = "Süre [Saniye]"

        # --- GRAFİK 1: VRM Sıcaklığı (Sol Üst) ---
        axes[0, 0].plot(seconds, df[col_vrm], label=name, linewidth=2, color=colors[i])
        
        # --- GRAFİK 2: PSU +12V Voltajı (Sağ Üst) ---
        axes[0, 1].plot(seconds, df[col_12v], label=name, linewidth=1.5, color=colors[i], alpha=0.9)
        
        # --- GRAFİK 3: Fan Eğrisi (Scatter: Sıcaklık vs RPM) (Sol Alt) ---
        # Bu grafik fanın nasıl tepki verdiğini gösterir
        # Yoğunluk karmaşasını önlemek için veriyi seyreltelim (her 5. veri)
        sample = df.iloc[::5, :]
        sns.scatterplot(x=sample[col_temp], y=sample[col_rpm], ax=axes[1, 0], 
                        color=colors[i], label=name, alpha=0.6, s=15)

        # --- İstatistik Hesaplama ---
        vrm_max = df[col_vrm].max()
        vrm_delta = vrm_max - df[col_vrm].iloc[0]
        
        v12_mean = df[col_12v].mean()
        v12_droop = df[col_12v].max() - df[col_12v].min() # Voltaj sarkması (V-Droop)
        
        fan_max = df[col_rpm].max()
        
        stats_list.append({
            'Profil': name,
            'Max VRM Isısı (°C)': vrm_max,
            'VRM Isınma Delta (°C)': vrm_delta,
            'Ort. +12V (V)': v12_mean,
            'V-Droop (Sarkma) [V]': v12_droop,
            'Max Fan Devri (RPM)': fan_max
        })

    # Düzenlemeler - Grafik 1 (VRM)
    axes[0, 0].set_title('Anakart VRM Isınma Analizi', fontsize=14)
    axes[0, 0].set_ylabel('MOSFET Sıcaklığı [°C]', fontsize=12)
    axes[0, 0].set_xlabel(x_label, fontsize=10)
    axes[0, 0].legend()
    
    # Düzenlemeler - Grafik 2 (PSU)
    axes[0, 1].set_title('PSU +12V Kararlılık Analizi', fontsize=14)
    axes[0, 1].set_ylabel('Voltaj [V]', fontsize=12)
    axes[0, 1].axhline(y=12.0, color='r', linestyle='--', alpha=0.5, label='İdeal 12V')
    # Voltaj grafiğinde dalgalanmayı görmek için Y eksenini daraltalım
    all_12v = pd.concat([df[col_12v] for df in data_dict.values() if df is not None])
    axes[0, 1].set_ylim(all_12v.min() - 0.05, all_12v.max() + 0.05)
    axes[0, 1].legend()

    # Düzenlemeler - Grafik 3 (Fan Curve)
    axes[1, 0].set_title('Soğutma Fiziği: Fan Tepki Eğrisi', fontsize=14)
    axes[1, 0].set_xlabel('İşlemci Sıcaklığı [°C]', fontsize=12)
    axes[1, 0].set_ylabel('Fan Hızı [RPM]', fontsize=12)
    axes[1, 0].grid(True, which='both', linestyle='--', linewidth=0.5)
    
    # Grafik 4: Isınma Hızı (Thermal Rate of Change)
    # Hangi profil işlemciyi daha yavaş ısıtıyor?
    # Türev (diff) alarak saniyelik sıcaklık artışını görebiliriz
    axes[1, 1].set_title('Anlık Isı Değişim Hızı (Thermal Shock)', fontsize=14)
    for i, (name, df) in enumerate(data_dict.items()):
        # Rolling mean ile gürültüyü azaltıp türev alalım
        temp_rate = df[col_temp].diff().rolling(window=10).mean()
        axes[1, 1].plot(temp_rate, label=name, color=colors[i], linewidth=1)
    
    axes[1, 1].set_ylabel('Delta T (°C / Örnek)', fontsize=12)
    axes[1, 1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    axes[1, 1].legend()

    plt.tight_layout()
    plt.show()
    
    # Özet Tablo
    print("\n--- Donanım Sağlığı ve Soğutma Özeti ---")
    stats_df = pd.DataFrame(stats_list).set_index('Profil')
    # Renklendirme: Düşük VRM, Düşük Droop, Düşük Delta iyidir.
    display(stats_df.style.background_gradient(cmap='RdYlGn_r', subset=['Max VRM Isısı (°C)', 'V-Droop (Sarkma) [V]', 'VRM Isınma Delta (°C)'])
                   .format("{:.3f}"))

analyze_system_health_and_cooling(dataframes)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

def analyze_correlations_and_uncore(data_dict):
    """
    Adım 13 ve 14: Değişkenler arası korelasyon ve Uncore (FCLK/UCLK/L3) analizi.
    """
    if not data_dict:
        return

    # --- 1. Sütun Seçimi ---
    # Analiz edilecek kritik parametreler
    # HWInfo isimleri değişiklik gösterebilir, anahtar kelimelerle bulalım
    target_metrics = {
        'Temp': ['CPU (Tctl/Tdie) [°C]'],
        'Clock': ['Çekirdek Etkili Saatler (avg) [MHz]', 'Effective Clock [MHz]'],
        'Voltage': ['Core VIDs (avg) [V]', 'CPU Core Voltage [V]'],
        'Power': ['CPU Tüm Güç Tüketimi [W]', 'CPU Package Power [W]'],
        'SoC Power': ['CPU SoC Güç Tüketimi (SVI3 TFN) [W]'],
        'Fan RPM': ['CPU [RPM]'],
        'FCLK': ['Infinity Fabric Saati (FCLK) [MHz]'],
        'UCLK': ['Bellek Denetleyici Saati (UCLK) [MHz]'],
        'L3 Temp': ['L3 Sıcaklıkları (avg) [°C]', 'L3 Cache (CCD1) [°C]']
    }
    
    # İlk df üzerinden sütunları kesinleştirelim
    first_df = next(iter(data_dict.values()))
    valid_cols = {}
    
    for key, candidates in target_metrics.items():
        for c in candidates:
            if c in first_df.columns:
                valid_cols[key] = c
                break
    
    print("Analiz Edilen Parametreler:", list(valid_cols.keys()))

    # --- 2. Korelasyon Matrisi (En Agresif Profil İçin) ---
    # Genelde en çok ilgilenilen profil "-30 CCD" veya "PBO" olandır.
    # İsmi içinde "-30" veya "PBO" geçen ilk profili seçelim, yoksa ilkini alalım.
    target_profile_name = next((name for name in data_dict.keys() if "-30" in name or "PBO" in name), list(data_dict.keys())[0])
    target_df = data_dict[target_profile_name]
    
    # Sadece sayısal sütunları içeren küçük bir DF oluşturalım
    corr_df = target_df[list(valid_cols.values())].rename(columns={v: k for k, v in valid_cols.items()})
    
    # Korelasyonu hesapla
    corr_matrix = corr_df.corr()

    # --- GÖRSELLEŞTİRME ---
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(2, 2)
    
    # Grafik 1: Korelasyon Isı Haritası
    ax_heatmap = fig.add_subplot(gs[0, 0])
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5, ax=ax_heatmap, center=0)
    ax_heatmap.set_title(f"Değişken İlişkileri (Korelasyon) - Profil: {target_profile_name}", fontsize=14)
    
    # Grafik 2: FCLK ve UCLK Kararlılığı (Uncore Stability)
    ax_uncore = fig.add_subplot(gs[0, 1])
    colors = sns.color_palette("tab10", n_colors=len(data_dict))
    
    if 'FCLK' in valid_cols and 'UCLK' in valid_cols:
        for i, (name, df) in enumerate(data_dict.items()):
            # Zaman ekseni (saniye)
            if isinstance(df.index, pd.DatetimeIndex):
                seconds = (df.index - df.index[0]).total_seconds()
            else:
                seconds = df.index
            
            # FCLK çiz
            ax_uncore.plot(seconds, df[valid_cols['FCLK']], label=f"{name} (FCLK)", color=colors[i], linestyle='-')
            # UCLK çiz (Kesikli çizgi) - Eğer üst üste binerse iyi demektir (Senkron)
            # Karmaşayı önlemek için sadece FCLK'yı çizelim, UCLK için farka bakalım
            
        ax_uncore.set_title('Infinity Fabric (FCLK) Kararlılığı', fontsize=14)
        ax_uncore.set_ylabel('Frekans [MHz]', fontsize=12)
        ax_uncore.legend(loc='lower right', fontsize=8)
        # Y eksenini daralt (Dalgalanmayı görmek için)
        all_fclk = pd.concat([df[valid_cols['FCLK']] for df in data_dict.values() if df is not None])
        ax_uncore.set_ylim(all_fclk.min() - 10, all_fclk.max() + 10)
    else:
        ax_uncore.text(0.5, 0.5, "FCLK/UCLK Verisi Bulunamadı", ha='center')

    # Grafik 3: L3 Cache Sıcaklığı vs Core Sıcaklığı
    # L3 önbellek, çekirdeklerden daha mı sıcak? (Hotspot Analysis)
    ax_l3 = fig.add_subplot(gs[1, :]) # Alt satırı kapla
    
    if 'L3 Temp' in valid_cols and 'Temp' in valid_cols:
        for i, (name, df) in enumerate(data_dict.items()):
            # Scatter plot yerine, Delta (L3 - Core) zaman serisi çizelim
            # Bu, L3'ün çekirdeklerden ne kadar daha sıcak/soğuk olduğunu gösterir
            l3_delta = df[valid_cols['L3 Temp']] - df[valid_cols['Temp']]
            
            # Zaman ekseni
            if isinstance(df.index, pd.DatetimeIndex):
                seconds = (df.index - df.index[0]).total_seconds()
            else:
                seconds = df.index

            ax_l3.plot(seconds, l3_delta, label=name, color=colors[i], linewidth=1.5, alpha=0.8)
            
        ax_l3.set_title('Hotspot Analizi: L3 Cache ile Çekirdek Sıcaklığı Farkı (Delta)', fontsize=14)
        ax_l3.set_ylabel('L3 Temp - Core Temp [°C]', fontsize=12)
        ax_l3.set_xlabel('Süre [Saniye]', fontsize=12)
        ax_l3.axhline(0, color='black', linestyle='--', alpha=0.5) # Eşitlik çizgisi
        ax_l3.legend()
        ax_l3.text(0.01, 0.9, "Pozitif Değer: L3 daha sıcak\nNegatif Değer: Çekirdek daha sıcak", 
                   transform=ax_l3.transAxes, bbox=dict(facecolor='white', alpha=0.7))

    plt.tight_layout()
    plt.show()

    # --- UCLK vs MCLK Senkronizasyon Kontrolü ---
    # Ryzen 7000 serisi için UCLK = MCLK idealdir (1:1 mode).
    # RAM'iniz 6000 MHz (3000 MHz MCLK) ise, UCLK da 3000 olmalı.
    if 'UCLK' in valid_cols:
        print("\n--- Bellek Kontrolcüsü (UCLK) Durumu ---")
        uclk_stats = []
        for name, df in data_dict.items():
            uclk_avg = df[valid_cols['UCLK']].mean()
            uclk_max = df[valid_cols['UCLK']].max()
            uclk_min = df[valid_cols['UCLK']].min()
            uclk_stats.append({
                'Profil': name,
                'Ort. UCLK (MHz)': uclk_avg,
                'Kararlılık (Max-Min)': uclk_max - uclk_min
            })
        display(pd.DataFrame(uclk_stats).set_index('Profil').style.format("{:.1f}"))

analyze_correlations_and_uncore(dataframes)